# Fantasy Football Draft Simulator Capstone Project
### By: John Crawford, Christopher Hinshaw, Jack Horgan

Package and Data Import

In [1]:
from operator import itemgetter
import pandas as pd
import random
import numpy as np
import scipy.stats as stats

In [2]:
QBs = pd.read_csv('qb_stats_and_projections.csv')
RBs = pd.read_csv('rb_stats_and_projections.csv')
WRs = pd.read_csv('wr_stats_and_projections.csv')
TEs = pd.read_csv('te_stats_and_projections.csv')

Data Cleaning and Preparation

In [3]:
#Naming discrepancies across different positions, rename to make them all match
QBs = QBs.rename(columns = {'2020 RUSH YDS': '2020 RUSH YARDS'})
RBs = RBs.rename(columns = {'2020 REC YDS': '2020 REC YARDS'})
WRs = WRs.rename(columns = {'2020 REC TDS': '2020 REC TD', '2020 RUSH TDS':'2020 RUSH TD', 'POSITION':'PLAYER POSITION', '2020 RUSH CAR': '2020 CAR', 'TEAM': 'TEAM NAME'})
TEs = TEs.rename(columns = {'2020 RUSH CAR': '2020 CAR', '2020 REC YDS': '2020 REC YARDS', '2020 RUSH YDS': '2020 RUSH YARDS'})

In [4]:
ffprojections = pd.concat([QBs, RBs, WRs, TEs]) #concatenate into one major dataframe

In [5]:
#focus on columns that are important to fantasy points
ffprojections = ffprojections[['PLAYER NAME', 'TEAM NAME', 
                                     'PLAYER POSITION', '2020 CAR', '2020 RUSH YARDS',
                                     '2020 RUSH TD', '2020 REC', '2020 REC YARDS', '2020 REC TD', 
                                     '2020 PASSING YDS', '2020 PASSING TD', '2020 INT']]

In [6]:
ffprojections = ffprojections.rename(columns = {'PLAYER NAME': 'Player', 'TEAM NAME': 'Tm', 'PLAYER POSITION':'Pos',
                                  '2020 CAR':'Car', '2020 RUSH YARDS': 'RushYards', '2020 RUSH TD': 'RushTD',
                                  '2020 REC': 'Rec', '2020 REC YARDS': 'RecYards', '2020 REC TD': 'RecTD',
                                  '2020 PASSING YDS': 'PassYards', '2020 PASSING TD': 'PassTD', '2020 INT': 'Int'}).fillna(0)

__Fantasy Point Scoring System__ <br>  
__QB__<br> 
    Pass Yard | 0.04 <br>
    Pass Touchdown | 4 <br>
    Interception | -1 <br>  
__RB__ <br> 
    Rush Yard | 0.1 <br>
    Rush Touchdown | 6 <br>  
__WR/TE__ <br> 
    Reception | 1 <br>
Recieving Yard | 0.1 <br>
    Receiving TD | 6 <br>


#### Generate Fantasy Point Datasets

In [7]:
#want as many random variables in our draft strategy analysis as possible
#randomize projections with multiplier to each stat from 0.925 to 1.075 
def rand_projections(ffprojections):

    rand_multiplier = 0.15 * np.random.random_sample((len(ffprojections), len(ffprojections.columns[3:]))) + 0.925
    
    #apply these scalar factors to numerical catefories
    ffprojections.iloc[:,3:] = ffprojections.iloc[:,3:] * rand_multiplier
    
    #compute projected fantasy points
    ffprojections['FPTS'] = ffprojections['RushYards'] * 0.1 + ffprojections['RushTD'] * 6 \
                        + ffprojections['Rec'] + ffprojections['RecYards'] * 0.1 + ffprojections['RecTD'] * 6 \
                        + ffprojections['PassYards'] * 0.04 + ffprojections['PassTD'] * 4 + ffprojections['Int'] * -1
    
    ffprojections = ffprojections[ffprojections['FPTS'] > 0] #clean players who aren't projected for any fantasy points
    ffprojections = ffprojections.sort_values('FPTS', ascending = False)
    
    #split back into positional dataframes to calculate positional z-scores
    RBs = ffprojections.loc[ffprojections['Pos'] == 'RB'].reset_index(drop = True)
    WRs = ffprojections.loc[ffprojections['Pos'] == 'WR'].reset_index(drop = True)
    TEs = ffprojections.loc[ffprojections['Pos'] == 'TE'].reset_index(drop = True)
    QBs = ffprojections.loc[ffprojections['Pos'] == 'QB'].reset_index(drop = True)
    
    #apply positional weights
    QBs['FPTS'] *= 0.8
    RBs['FPTS'] *= 1.02
    WRs['FPTS'] *= 0.98
    TEs['FPTS'] *= 1.1
    
    #calculate new column with z-scores for each position
    def z_score(df):
        df['Zscore'] = (df.loc[:, 'FPTS'] - df['FPTS'].mean()) / df['FPTS'].std()
        return df
    RBs = z_score(RBs)
    WRs = z_score(WRs)
    TEs = z_score(TEs)
    QBs = z_score(QBs)
    
    return QBs, RBs, WRs, TEs

In [8]:
QBs, RBs, WRs, TEs = rand_projections(ffprojections)

In [9]:
QBs.head(15) #check dataset

,Player,Tm,Pos,Car,RushYards,RushTD,Rec,RecYards,RecTD,PassYards,PassTD,Int,FPTS,Zscore
0,Lamar Jackson,Ravens,QB,147.434907,943.062397,6.423049,0.0,0.0,0.0,3354.917831,24.519284,10.714126,283.523404,1.657563
1,Patrick Mahomes,Chiefs,QB,63.334382,259.669245,2.811841,0.0,0.0,0.0,4723.563022,30.256690,10.259886,274.037893,1.563996
2,Deshaun Watson,Texans,QB,88.130508,549.763502,3.857264,0.0,0.0,0.0,4214.743698,26.356320,11.714967,272.335999,1.547208
3,Dak Prescott,Cowboys,QB,49.497906,232.393017,4.221296,0.0,0.0,0.0,4218.240130,27.624988,11.438471,253.086531,1.357328
4,Kyler Murray,Cardinals,QB,81.186015,467.689854,2.820539,0.0,0.0,0.0,3888.150812,27.325858,12.314963,252.965378,1.356132
5,Russell Wilson,Seahawks,QB,70.690090,373.232105,2.089691,0.0,0.0,0.0,3732.202865,30.084167,9.663933,247.857764,1.305750
6,Tom Brady,Buccaneers,QB,23.045516,28.551789,1.857863,0.0,0.0,0.0,4589.824585,28.855578,9.067982,243.159739,1.259408
7,Matt Ryan,Falcons,QB,31.296383,121.517235,1.054701,0.0,0.0,0.0,4563.206481,28.600808,11.862800,242.838898,1.256243
8,Carson Wentz,Eagles,QB,52.663621,202.373662,2.141215,0.0,0.0,0.0,3698.562476,28.714308,8.743511,229.712702,1.126763
9,Daniel Jones,Giants,QB,51.671173,271.201593,2.109694,0.0,0.0,0.0,3940.539978,24.336642,13.796474,224.760017,1.077909


### The Draft

Our Draft Strategy: Take highest z-score difference between first and second players for a position <br>
Autodraft Strategy: Take highest projected player across all positions

In [10]:
#This class stores the team information for all 12 teams in the fantasy draft
class Team:
    def __init__(self):
        #variable to store a list of the selected players for a team
        self.team = []

        #maximum amount of players that can be selected from one position
        self.player_types = {
            'QB': 2,
            'RB': 6,
            'WR': 8,
            'TE': 1
        }
        
    #function to draft players for our team
    def OurDraft(self):
        # Dictionary with possible choices; z-score difference as key and position as value
        choices = {QBs['Zscore'].iloc[0] - QBs['Zscore'].iloc[1]: [QBs, 'QB'],
                           RBs['Zscore'].iloc[0] - RBs['Zscore'].iloc[1]: [RBs, 'RB'],
                           WRs['Zscore'].iloc[0] - WRs['Zscore'].iloc[1]: [WRs, 'WR'],
                           TEs['Zscore'].iloc[0] - TEs['Zscore'].iloc[1]: [TEs, 'TE'] }
        
        #sort dictionary by greatest positional differential
        choices_sorted = dict(sorted(choices.items(), reverse = True))

        for x in choices_sorted.keys(): #cycle through values in dictionary in order of difference
            position_drafted = choices_sorted[x]
            if self.player_types[position_drafted[1]] > 0: #if we already have the max for this position, cycle to next highest difference
                self.team.append([position_drafted[0]['Player'].iloc[0], position_drafted[1]])  # 'draft' player
                position_drafted[0].drop(0, inplace=True) #drop from dataset because we 'drafted'
                position_drafted[0].reset_index(inplace=True, drop=True)
                self.player_types[position_drafted[1]] -= 1 #decrease max players for the position selected
                break #break out of loop once player has been drafted

    def AutoDraft(self):
        #QBs project for way more points than other positions, but since each team can only start one, they aren't as valuable
        #Apply positional weighting to projected fantasy points to get a more realistic and balanced draft

        
        #same code as OurDraft except they take top projected player overall
        choices =  {QBs['FPTS'].iloc[0] : [QBs, 'QB'],
                       RBs['FPTS'].iloc[0] : [RBs, 'RB'],
                       WRs['FPTS'].iloc[0] : [WRs, 'WR'],
                       TEs['FPTS'].iloc[0] : [TEs, 'TE'] }
        choices_sorted = dict(sorted(choices.items(), reverse = True))
        for x in choices_sorted.keys():
            position_drafted = choices_sorted[x]
            if self.player_types[position_drafted[1]] > 0: 
                self.team.append([position_drafted[0]['Player'].iloc[0], position_drafted[1]])
                position_drafted[0].drop(0, inplace=True)
                position_drafted[0].reset_index(inplace=True, drop=True)
                self.player_types[position_drafted[1]] -= 1 
                break 

    def get_team(self, sort = True):
        
        #sort by position
        if sort:
            return sorted(self.team, key=itemgetter(1))
        
        #sort by draft order
        else:
            return self.team

    def __str__(self):
        return str(self.team)

__Positions__: <br>  
QB <br>
RB <br>
RB <br>
WR <br>
WR <br>
TE <br>
FLEX (WR/RB/TE) <br>
BENCH x10 <br>  
17 total players


In [11]:
#function that automates a draft with inputted draft position for our team
def draft(draft_positon):
 #, QBs, RBs, WRs, TEs
    draft_board = [Team(), #initalize a draft board with 12 different teams
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team(),
        Team()]

    num_rounds = 17 #number of rounds in the draft

    for i in range(0, num_rounds): #cycle through each round of a draft
        if not (i % 2): #if draft round is odd, normal order
            for j in range(0, 12):
                if j+1 == draft_positon:
                    draft_board[j].OurDraft()
                else:
                    draft_board[j].AutoDraft()
        else: #if draft round is even, reverse order (snake draft)
            for j in range(11, -1, -1):
                if j+1 == draft_positon:
                    draft_board[j].OurDraft()
                else:
                    draft_board[j].AutoDraft()
                    
    #Create dataframe to display draft board
    data = {}
    for x in range(0,12):
        if x+1 == draft_positon: 
            data['My Team'] = draft_board[x].get_team(sort = False)
        else: 
            data[f'Team {x + 1}'] = draft_board[x].get_team(sort = False)
    Draft = pd.DataFrame(data = data, index = range(1,num_rounds + 1))
    return Draft



In [32]:
draft_positon = random.randint(1, 12) #randomize spot to select
QBs, RBs, WRs, TEs = rand_projections(ffprojections)
Draft = draft(draft_positon)

In [33]:
Draft

,My Team,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8,Team 9,Team 10,Team 11,Team 12
1,"[Travis Kelce, TE]","[Cooper Kupp, WR]","[Nick Chubb, RB]","[Patrick Mahomes, QB]","[Alvin Kamara, RB]","[Michael Thomas, WR]","[Kyler Murray, QB]","[Daniel Jones, QB]","[A.J. Brown, WR]","[Marvin Jones Jr., WR]","[Josh Allen, QB]","[George Kittle, TE]"
2,"[Dak Prescott, QB]","[Tyreek Hill, WR]","[Russell Wilson, QB]","[Lamar Jackson, QB]","[David Johnson, RB]","[Leonard Fournette, RB]","[Christian McCaffrey, RB]","[DeAndre Hopkins, WR]","[Sam Darnold, QB]","[Ezekiel Elliott, RB]","[Odell Beckham Jr., WR]","[Amari Cooper, WR]"
3,"[Josh Jacobs, RB]","[Jamison Crowder, WR]","[Davante Adams, WR]","[Le'Veon Bell, RB]","[Dalvin Cook, RB]","[Ke'Shawn Vaughn, RB]","[Mike Evans, WR]","[Dwayne Haskins Jr., QB]","[Derek Carr, QB]","[James Conner, RB]","[D'Andre Swift, RB]","[A.J. Green, WR]"
4,"[Aaron Rodgers, QB]","[Drew Brees, QB]","[Tom Brady, QB]","[T.Y. Hilton, WR]","[Jared Goff, QB]","[Jimmy Garoppolo, QB]","[Ryan Tannehill, QB]","[DK Metcalf, WR]","[Calvin Ridley, WR]","[Kirk Cousins, QB]","[Rob Gronkowski, TE]","[Joe Mixon, RB]"
5,"[Derrick Henry, RB]","[Chris Carson, RB]","[Kenyan Drake, RB]","[Melvin Gordon, RB]","[Devin Singletary, RB]","[Noah Fant, TE]","[DJ Chark Jr., WR]","[Curtis Samuel, WR]","[Austin Hooper, TE]","[Matt Ryan, QB]","[Marquise Brown, WR]","[Brandin Cooks, WR]"
6,"[Saquon Barkley, RB]","[Jordan Howard, RB]","[Kerryon Johnson, RB]","[Aaron Jones, RB]","[Allen Robinson II, WR]","[Will Fuller V, WR]","[Todd Gurley II, RB]","[Evan Engram, TE]","[Adam Thielen, WR]","[Tyler Higbee, TE]","[Miles Sanders, RB]","[Mecole Hardman, WR]"
7,"[Christian Kirk, WR]","[Tyler Boyd, WR]","[Phillip Lindsay, RB]","[Terry McLaurin, WR]","[Austin Ekeler, RB]","[Courtland Sutton, WR]","[Darren Waller, TE]","[Derrius Guice, RB]","[DJ Moore, WR]","[Robby Anderson, WR]","[Teddy Bridgewater, QB]","[Deshaun Watson, QB]"
8,"[Chris Godwin, WR]","[Diontae Johnson, WR]","[Kenny Golladay, WR]","[Zach Ertz, TE]","[Dallas Goedert, TE]","[Joe Burrow, QB]","[Deebo Samuel, WR]","[Stefon Diggs, WR]","[Darius Slayton, WR]","[John Brown, WR]","[Breshad Perriman, WR]","[Kareem Hunt, RB]"
9,"[James Washington, WR]","[Ben Roethlisberger, QB]","[Raheem Mostert, RB]","[Clyde Edwards-Helaire, RB]","[Denzel Mims, WR]","[JuJu Smith-Schuster, WR]","[Michael Gallup, WR]","[Tarik Cohen, RB]","[Tyler Lockett, WR]","[Sterling Shepard, WR]","[Julio Jones, WR]","[Tua Tagovailoa, QB]"
10,"[Josh Reynolds, WR]","[Hunter Henry, TE]","[Mark Andrews, TE]","[Jalen Richard, RB]","[Gardner Minshew II, QB]","[Randall Cobb, WR]","[Duke Johnson, RB]","[DeVante Parker, WR]","[Damien Williams, RB]","[Anthony Miller, WR]","[Albert Wilson, WR]","[Robert Woods, WR]"


### Creating Head-to-Head Weekly Matchups and Calculating Record

In [14]:
#Adds up the points from the players for each week for each team
class lineUp:
    def __init__(self, team, week, team_name):
        self.team = team #same team list from Team() class
        self.week = week #week number (1-17)
        self.team_name = team_name
        self.line_up = pd.DataFrame(columns = [ 'Player', 'Pos', 'FPTS']) #fill out empty dataframe with starting players for each week

    def getPoints(self):
        
        #Intialize dataframe with team list
        points = pd.DataFrame(self.team)
        
        #Split lists into player and position columns
        points[['Player','Pos']] = pd.DataFrame(points[self.team_name].tolist(), index= points.index)
        points = points.drop(self.team_name, axis = 1)
        
        #match up our team with weekly scoring dataframe through merge (way faster than iteration)
        points = points.merge(weekly_data[week], on = ['Player', 'Pos'], how = 'left')
        points['FPTS'] = points['PPRFantasyPoints'].fillna(0)
        points = points.drop('PPRFantasyPoints', axis = 1)
        return points
    
    #Now that we have all our players with their point values, need to set starting lineup
    #Starting lineup automatically optimized after weekly scoring -- best ball format, no setting lineups
    def getLineUp(self, points):
        
        #sort each position by fantasy points scored
        QBs = points[points['Pos'] == 'QB'].sort_values('FPTS', ascending = False)
        RBs = points[points['Pos'] == 'RB'].sort_values('FPTS', ascending = False)
        WRs = points[points['Pos'] == 'WR'].sort_values('FPTS', ascending = False)
        TEs = points[points['Pos'] == 'TE'].sort_values('FPTS', ascending = False)
        
        self.line_up.loc[len(self.line_up.index)] = QBs.iloc[0] #start top scoring QB
        
        self.line_up.loc[len(self.line_up.index)] = RBs.iloc[0] #start top scoring RB
        
        self.line_up.loc[len(self.line_up.index)] = RBs.iloc[1] #start second highest scoring RB

        self.line_up.loc[len(self.line_up.index)] = WRs.iloc[0] #start top scoring WR
        
        self.line_up.loc[len(self.line_up.index)] = WRs.iloc[1] #start second highest scoring WR
        
        self.line_up.loc[len(self.line_up.index)] = TEs.iloc[0] #start top scoring TE
        
        #still need to fill out flex position (any player except a QB)
        #combine remaining players into dataset
        Flex = pd.concat([RBs.iloc[2: , :], WRs.iloc[2: , :], TEs.iloc[1: , :]]).sort_values('FPTS', ascending = False)
        
        self.line_up.loc[len(self.line_up.index)] = Flex.iloc[0] #start highest scoring remaining non-QB
        
        #display positions - no column name for aesthetic purposes when printing dataframe
        self.line_up[' '] = ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'TE', 'Flex']
        self.line_up.set_index(' ', inplace = True)

    def printLineUp(self):
        #print the team name, the starting lineup, and the total fantasy points scored
        print(f'\n \033[1m {self.team_name}: \033[0m \n {self.line_up} \n \n Total Points: {self.getTotal()} \n')

    def getTotal(self):
        return round(sum(self.line_up.FPTS), 2)



In [15]:
#randomize schedule of opponents
#play every team once, then play remaining weeks with random teams
def random_opponents(draft_position, num_weeks):
    if num_weeks < 12:
        raise Exception('Season is too short! Need at least 12 weeks')
    opponent_list = []
    for x in range(1, 13):
        if x != draft_position:
            opponent_list.append(x)
    n = num_weeks - 11
    while n > 0:
        rand_team = random.randint(1,12)
        while rand_team == draft_position: #if random team is our team, keep randomizing until it isn't
            rand_team = random.randint(1,12)
        opponent_list.append(rand_team)
        n -= 1
        
    #randomize entire schedule
    random.shuffle(opponent_list)

    return opponent_list

In [16]:
random_opponents(4, 13) #first parameter our team's draft position, second paramter # of weeks to make a schedule for

[3, 5, 6, 8, 12, 1, 2, 7, 10, 9, 8, 11, 8]

Note: If the code below is taking a long time to run, it is most likely an issue with the github datasource not loading - usually works after a bit

In [17]:
#load in weekly scoring dataframe in a dictionary: week # as key and data for that week as values
weekly_data = {}
for x in range(1, 17):
    temp = pd.read_csv(f'https://raw.githubusercontent.com/fantasydatapros/data/master/weekly/2020/week{x}.csv') #scrape data
    temp = temp.loc[temp['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]
    weekly_data[x] = temp[['Player', 'Pos','PPRFantasyPoints']]

In [18]:
#17 weeks in football season, most fantasy leagues only play through first 16 because many players rest during week 17
num_weeks = 16


opponents_list = random_opponents(draft_positon, num_weeks) #draft_position variable from earlier
opponents = {}
for x, i in enumerate(opponents_list):
    opponents[x] = Draft[f'Team {i}']

my_team_totals = []
opponent_totals = []

for week in range(1, num_weeks + 1): #cycle through each week
    my_team_lineup = lineUp(Draft['My Team'], week, 'My Team') #initialize class
    my_team_lineup.getLineUp(my_team_lineup.getPoints())
    print(f'-----\tWeek {week}\t-----') #print week number
    my_team_lineup.printLineUp() #print our team and score
    my_team_totals.append(my_team_lineup.getTotal()) #append to list to calculate record
    
    opponent_lineup = lineUp(opponents[week - 1], week, f'Team {opponents_list[week - 1]}') #initalize class for opponent team
    opponent_lineup.getLineUp(opponent_lineup.getPoints())
    opponent_lineup.printLineUp()
    opponent_totals.append(opponent_lineup.getTotal())

wins = 0
losses = 0

#calculate record
for week in range(num_weeks):
    if my_team_totals[week] > opponent_totals[week]:
        wins += 1
    else:
        losses += 1

print('---------------------')
print(f'Record: {wins} - {losses}')
print(f'Average Points: {sum(my_team_totals) / num_weeks}')


-----	Week 1	-----

  My Team:  
                   Player Pos   FPTS
                                   
QB    Gardner Minshew II  QB  20.82
RB1        David Johnson  RB   19.9
RB2        Melvin Gordon  RB   15.6
WR1        Sammy Watkins  WR   21.5
WR2        Mike Williams  WR   10.9
TE             Zach Ertz  TE   10.8
Flex     Jonathan Taylor  RB   14.9 
 
 Total Points: 114.42 


  Team 7:  
                Player Pos   FPTS
                                
QB     Deshaun Watson  QB  20.82
RB1     Chase Edmonds  RB   13.5
RB2        Nick Chubb  RB    5.6
WR1    Marquise Brown  WR   15.1
WR2   Diontae Johnson  WR    9.7
TE      Darren Waller  TE   10.5
Flex      Jerry Jeudy  WR    9.6 
 
 Total Points: 84.82 

-----	Week 2	-----

  My Team:  
                   Player Pos   FPTS
                                   
QB    Gardner Minshew II  QB  23.46
RB1      Jonathan Taylor  RB   19.0
RB2        Austin Ekeler  RB   18.8
WR1         Keenan Allen  WR   16.6
WR2       Adam Humphries  WR

## Simulating Results

Can't just use one season as indication of how good our strategy is, need to evaluate win percentage over 'n' simulations <br>  
__Change n value on first line below to a lower number if you want this to run quickly__

In [19]:
n= 100 # number of simulations 

num_weeks = 16 # number of weeks per simulation
win_list = np.array([]) #record wins for each season
loss_list = np.array([]) #record losses for each season

while n > 0:    
    QBs, RBs, WRs, TEs = rand_projections(ffprojections)


    draft_positon = random.randint(1, 12) #randomize spot to select

    Draft = draft(draft_positon)


    opponents_list = random_opponents(draft_positon, num_weeks)
    opponents = {}
    for x, i in enumerate(opponents_list):
        opponents[x] = Draft[f'Team {i}']

    my_team_totals = []
    opponent_totals = []

    for week in range(1, num_weeks + 1):
        my_team_lineup = lineUp(Draft['My Team'], week, 'My Team')
        my_team_lineup.getLineUp(my_team_lineup.getPoints())
        my_team_totals.append(my_team_lineup.getTotal())
        opponent_lineup = lineUp(opponents[week - 1], week, f'Team {opponents_list[week - 1]}')
        opponent_lineup.getLineUp(opponent_lineup.getPoints())
        opponent_totals.append(opponent_lineup.getTotal())

    wins = 0
    losses = 0
    for week in range(num_weeks):
        if my_team_totals[week] > opponent_totals[week]:
            wins += 1
        else:
            losses += 1
    win_list = np.append(win_list, [wins])
    loss_list = np.append(loss_list, [losses])
    if n % 5 == 0:
        print(f'Simulations Left: {n}')
    n -= 1
print('Simulations Complete! win_list and loss_list can now be used')

Simulations Left: 100
Simulations Left: 95
Simulations Left: 90
Simulations Left: 85
Simulations Left: 80
Simulations Left: 75
Simulations Left: 70
Simulations Left: 65
Simulations Left: 60
Simulations Left: 55
Simulations Left: 50
Simulations Left: 45
Simulations Left: 40
Simulations Left: 35
Simulations Left: 30
Simulations Left: 25
Simulations Left: 20
Simulations Left: 15
Simulations Left: 10
Simulations Left: 5
Simulations Complete! win_list and loss_list can now be used


In [20]:
win_percentage = win_list.mean() / (win_list.mean() + loss_list.mean()) 
win_percentage

0.49875

Compare Our Simulated Average Wins with expected wins, num_weeks / 2 (50% win rate) <br>
Use one-sample T-test and derive p-value <br>
If p-value is greater than our selected alpha value of 0.05, then it is significant

In [21]:
p_value = stats.ttest_1samp(win_list, popmean= (num_weeks / 2)).pvalue
print(f'p_value: {p_value} \n')
if win_percentage < 0.5 and p_value > 0.05:
    print('Our Draft Strategy is not significantly better than AutoDraft!')
else:
    print('Our Draft Strategy is better than AutoDraft!')

p_value: 0.9468311560277882 

Our Draft Strategy is not significantly better than AutoDraft!
